# Runnable sequence

In [1]:
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
import os

In [3]:
load_dotenv()
API_KEY=os.getenv("API_KEY")
api=os.getenv('mapi')

In [2]:
from langchain.schema.runnable import RunnableSequence


In [4]:
llm = ChatGoogleGenerativeAI(
    model ="gemini-2.5-flash",
    google_api_key = API_KEY
)

In [5]:
prompt=PromptTemplate(
    template="Tell me a joke {joke}",
    input_variables=['joke']
)
parser =StrOutputParser()
chain= RunnableSequence(prompt,llm,parser)

In [7]:
print(chain.invoke({"funny"}))

Why don scientists not trust atoms?

Because they make up everything!


In [8]:
prompt1=PromptTemplate(
    template="Explain the joke {text}",
    input_variables=['text']
)
parser =StrOutputParser()
chain= RunnableSequence(prompt,llm,parser,prompt1,llm,parser)

In [9]:
print(chain.invoke({"funny"}))

This joke plays on the double meaning of the phrase "make up everything":

1.  **Literal/Scientific Meaning:** In science, atoms are indeed the fundamental building blocks that **constitute** or **form** everything in the universe. Everything is literally "made up of" atoms.

2.  **Figurative/Colloquial Meaning (the pun):** To "make up" something can also mean to **invent, fabricate, or lie about something**. For example, "He made up a story" means he lied.

The humor comes from applying the *second* meaning (lying/fabricating) to atoms, even though the *first* meaning (being the building blocks of everything) is the scientific truth. If atoms "make up" everything in the sense of "lying about everything," then you certainly wouldn't trust them!


# Runnable parallel

In [11]:
from langchain_mistralai.chat_models import ChatMistralAI

llm= ChatGoogleGenerativeAI(
    model ="gemini-2.5-flash",
    google_api_key = API_KEY
)
llm2=ChatMistralAI(
    model="mistral-small-latest",  
    mistral_api_key=api,
)


In [12]:
from langchain.schema.runnable import RunnableParallel

prompt1= PromptTemplate(
    template="Genarate tweet about topic {topic}",
    input_variables=['topic']
)
prompt2= PromptTemplate(
    template="Genarate linkedin post about topic {topic}",
    input_variables=['topic']
)
parser=StrOutputParser()
parallel=RunnableParallel({
    'tweet':RunnableSequence(prompt1,llm,parser),
    'post':RunnableSequence(prompt2,llm,parser)
})

In [14]:
result=parallel.invoke({"topic":"AI"})

In [17]:
print(result['tweet'])


Here are a few options for a tweet about AI, covering different angles:

**Option 1 (Optimistic & Forward-Looking):**
> The future is now! 🤖 AI is revolutionizing industries, from healthcare to education, and unlocking new possibilities. What innovation excites you most? ✨ #AI #ArtificialIntelligence #FutureTech

**Option 2 (Thought-Provoking & Ethical):**
> As AI rapidly advances, so do the conversations around ethics, bias, and job displacement. How do we ensure a responsible and equitable AI future? 🤔 Share your thoughts! #AIEthics #FutureOfWork

**Option 3 (Concise & Engaging Question):**
> AI is everywhere! From recommending your next show to powering self-driving cars. What's one surprising way AI has impacted your life recently? Share below! 👇 #AIImpact

**Option 4 (Focus on Transformation):**
> The AI revolution is in full swing! 🧠 From automating tasks to solving complex problems, artificial intelligence is reshaping our world at an incredible pace. #TechInnovation #DigitalTra

In [18]:
print(result['post'])

Here are a few options for a LinkedIn post about AI, catering to different angles and tones. Choose the one that best fits your personal brand and message!

---

**Option 1: The Optimistic & Future-Focused Post**

🚀 **AI isn't just a buzzword; it's the engine driving the next wave of innovation across every industry.**

From streamlining operations to unlocking unprecedented insights and personalizing experiences, Artificial Intelligence is reshaping how we work, live, and create. It's no longer a futuristic concept but a powerful tool available today.

The key now is not just *if* we adopt AI, but *how* we integrate it responsibly and strategically to augment human potential.

What industry do you believe AI will revolutionize most in the next 5 years, and why? Share your thoughts below!

#AI #ArtificialIntelligence #Innovation #FutureOfWork #DigitalTransformation #TechTrends

---

**Option 2: The Practical & Skill-Oriented Post**

Feeling overwhelmed by the rapid pace of AI advanceme

# Runnable Pass through
- return input as ouput without modifying
- useful for testing, skipping steps, or keeping workflow uniform.

In [23]:
from langchain.schema.runnable import RunnablePassthrough

prompt=PromptTemplate(
    template="Tell me a joke {joke}",
    input_variables=['joke']
)
parser =StrOutputParser()
chain= RunnableSequence(prompt,llm,parser)

In [27]:
prompt1=PromptTemplate(
    template="Explain the joke {text}",
    input_variables=['text']
)
parser =StrOutputParser()
j_chain= RunnableParallel({
    "joke":RunnablePassthrough(),
    "Explation": RunnableSequence(prompt1,llm,parser)
    }
)

In [28]:
final_chain=RunnableSequence(chain,j_chain)

In [30]:
print(final_chain.invoke({"joke":"Science"}))

{'joke': 'Why did the atom get pulled over?\n\nBecause it had a **charge**!', 'Explation': 'This joke is a pun that plays on the double meaning of the word "charge"!\n\nHere\'s the breakdown:\n\n1.  **Meaning 1 (Legal/Common Usage):** When a person (or a car) gets "pulled over" by the police, it\'s often because they\'ve committed an offense, and they might face a **legal charge** (an accusation of a crime, e.g., "charged with speeding" or "charged with reckless driving"). This is the expected meaning when someone is pulled over.\n\n2.  **Meaning 2 (Scientific/Physics):** In science, an **atom** can have an **electrical charge**.\n    *   Atoms are made of protons (positive charge), neutrons (no charge), and electrons (negative charge).\n    *   If an atom gains or loses electrons, it becomes an ion and has an overall positive or negative "charge." A neutral atom has no net charge.\n\nThe humor comes from the unexpected twist: you expect the answer to be a legal "charge" because it was

# Runnable Lambda
- you wrap any simple Python function (or lambda) so it can be treated like a Runnable.

In [31]:
from langchain.schema.runnable import RunnableLambda

prompt=PromptTemplate(
    template="Tell me a joke {joke}",
    input_variables=['joke']
)
parser =StrOutputParser()
chain= RunnableSequence(prompt,llm,parser)

In [ ]:
def word_count(text):
    return len(text.split())
""" 
x = "apple,banana,orange"
x.split(",")  # Output: ['apple', 'banana', 'orange']

len("Hello")       # Output: 5 (characters)
len(["a","b","c"]) # Output: 3 (elements)
"""

In [ ]:

parser =StrOutputParser()
j_chain= RunnableParallel({
    "joke":RunnablePassthrough(),
    "length": RunnableLambda(word_count) # can be RunnableLambda (lambda x :x len(x.split()))
    }
)

In [35]:
final_chain=RunnableSequence(chain,j_chain)

In [37]:
result= final_chain.invoke({'joke':'AI'})

In [38]:
final_result="""{}\nword count {}""".format (result['joke'],result['length'])

In [40]:
print(final_result)

Okay, here's one for you!

Why did the scarecrow win an award?
... Because he was outstanding in his field!
word count 20


# Conditional Runnable

In [47]:
from langchain.schema.runnable import RunnableBranch

prompt=PromptTemplate(
    template="write a detail in a topic {topic}",
    input_variables=['topic']
)
prompt2=PromptTemplate(
    template="Summarize the following text{text}",
    input_variables=['text']
)


In [48]:
report_gen_chain=RunnableSequence(prompt,llm,parser)


In [52]:
branch_chain=RunnableBranch(
    (lambda x: len(x.split())>200, RunnableSequence(prompt2,llm,parser)),
    RunnablePassthrough()
)

In [53]:
final_chain=RunnableSequence(report_gen_chain,branch_chain)

In [54]:
print(final_chain.invoke({"topic":"WW2"}))

The Battle of Stalingrad (August 1942 – February 1943) was a pivotal and brutal turning point on World War II's Eastern Front, shattering the myth of German invincibility.

Germany's objective (Operation Blau) was to capture the oil-rich Caucasus and secure Stalingrad, a strategic industrial city on the Volga River, which also held symbolic importance bearing Stalin's name. The battle began with a devastating Luftwaffe bombing on August 23, 1942, reducing the city to rubble, which paradoxically made subsequent urban combat, dubbed "Rattenkrieg" (Rat War), even more horrific. Fighting devolved into brutal house-to-house, room-to-room combat, particularly around key industrial complexes, resulting in immense casualties for both sides. German Panzer divisions were largely ineffective in the ruins.

While the Germans were bogged down, Soviet Marshal Georgy Zhukov meticulously planned "Operation Uranus," a massive pincer movement launched on November 19, 1942. This operation swiftly encircl